TAREA: Captura una o varias imágenes con monedas no solapadas. Tras visualizar la imagen, identifica de forma interactiva (por ejemplo haciendo clic en la imagen) una moneda de un valor determinado en la imagen (por ejemplo de 1€). Tras ello, la tarea se resuelve mostrando por pantalla el número de monedas y la cantidad de dinero presentes en la imagen. No hay restricciones sobre utilizar medidas geométricas o de color. ¿Qué problemas han observado?

Nota: Para establecer la correspondencia entre píxeles y milímetros, comentar que la moneda de un euro tiene un diámetro de 23.25 mm. la de 50 céntimos de 24.35, la de 20 céntimos de 22.25, etc. 

Extras: Considerar que la imagen pueda contener objetos que no son monedas y/o haya solape entre las monedas. Demo en vivo.

Importación de paquetes

In [151]:
import cv2  
import numpy as np
import matplotlib.pyplot as plt

moneCalculate(), función que se encarga de calcular el dinero total de la imagen

In [152]:
def moneyCalculate(coins):
    return coins[0]*0.01 + coins[1]*0.02 + coins[2]*0.05 + coins[3]*0.1 + coins[4]*0.2 + coins[5]*0.5 + coins[6]*1 + coins[7]*2

Lectura de imágenes y variables globales

In [153]:
# Read pictures
imagen = cv2.imread("Monedas.jpg")
monedas = [1,1,1,1,1,1,1,1]

imagen_p1 = cv2.imread("VC - Prueba-Monedas1.jpg")
monedas_p1 = [3,3,1,1,1,1,0,0]

imagen_p2 = cv2.imread("VC - Prueba-Monedas2.jpg")
monedas_p2 = [3,4,2,1,1,1,2,0]

img = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)
img_rgb_p1 = cv2.cvtColor(imagen_p1, cv2.COLOR_BGR2RGB)
img_rgb_p2 = cv2.cvtColor(imagen_p2, cv2.COLOR_BGR2RGB)

# Global variable
coins = [0] * 8
circulos_filtrados = []
diametros_reales = {
    0.01: 16.25,  # Moneda de 1 céntimo
    0.02: 18.75,  # Moneda de 2 céntimos
    0.05: 21.25,  # Moneda de 5 céntimos
    0.1: 19.75,   # Moneda de 10 céntimos
    0.2: 22.25,   # Moneda de 20 céntimos
    0.5: 24.25,   # Moneda de 50 céntimos (referencia)
    1.0: 23.25,   # Moneda de 1 euro
    2.0: 25.75    # Moneda de 2 euros
}


Función que cambia el fondo a blanco.

In [154]:
def convertir_fondo_a_blanco(imagen):
    # Convert the image to HSV color space to create a mask
    img_hsv = cv2.cvtColor(imagen, cv2.COLOR_BGR2HSV)
    
    # Define the color range to detect coins in HSV
    lower_bound = np.array([0, 30, 40])    # Lower limit
    upper_bound = np.array([50, 255, 255]) # Upper limit
    
    # Create a mask that selects only the coins
    mascara = cv2.inRange(img_hsv, lower_bound, upper_bound)
    
    # Perform a morphological closing operation to join areas and improve coin detection
    kernel = np.ones((5, 5), np.uint8)
    mascara_cerrada = cv2.morphologyEx(mascara, cv2.MORPH_CLOSE, kernel)
    
    # Detect contours in the mask
    contornos, _ = cv2.findContours(mascara_cerrada, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create an empty mask where only large contours (coins) will be kept
    mascara_filtrada = np.zeros_like(mascara_cerrada)
    
    # Filter contours by area, keeping only those large enough to be coins
    for contorno in contornos:
        area = cv2.contourArea(contorno)
        
        # Filter small fragments: adjust the minimum area as needed
        if area > 1000:  # Keep only large contours (coins)
            cv2.drawContours(mascara_filtrada, [contorno], -1, 255, -1)  # Fill the contour in the filtered mask

    # Apply the filtered mask on the original image to change the background to white
    imagen_blanco_fondo = imagen.copy()
    imagen_blanco_fondo[mascara_filtrada == 0] = [255, 255, 255]  # Assign white to areas outside of the coins

    return imagen_blanco_fondo

Función que detecta círculos dentro de la imagen.

In [155]:
def detectar_circulos(imagen):
    # Convert the image to grayscale
    imagen_gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    
    # Apply blur to reduce noise and improve edge detection
    imagen_gris = cv2.medianBlur(imagen_gris, 5)
    
    # Detect circles with HoughCircles
    circulos = cv2.HoughCircles(imagen_gris, 
                                cv2.HOUGH_GRADIENT, 
                                dp=1.01, 
                                minDist=150, 
                                param1=100, 
                                param2=25, 
                                minRadius=15, 
                                maxRadius=110)
    
    # List to store the detected circles
    circulos_detectados = []
    
    if circulos is not None:
        # Round and convert detected circles to integers
        circulos = np.round(circulos[0, :]).astype("int")
        
        # Store each circle in the list
        for (x, y, r) in circulos:
            circulos_detectados.append((x, y, r))
    
    return circulos_detectados


Funcion para filtrar los circulos.

In [156]:
def filtrar_por_color(imagen, circulos, rango_color_min, rango_color_max):
    # Convert the image to HSV
    imagen_hsv = cv2.cvtColor(imagen, cv2.COLOR_BGR2HSV)
    circulos_filtrados = []

    for (x, y, r) in circulos:
        # Create a mask for the current circle
        mascara = np.zeros(imagen.shape[:2], dtype=np.uint8)
        cv2.circle(mascara, (x, y), r, 255, -1)

        # Calculate the average color within the circle in HSV
        color_promedio = cv2.mean(imagen_hsv, mask=mascara)[:3]

        # Check if the average color is within the range
        if (rango_color_min[0] <= color_promedio[0] <= rango_color_max[0] and
            rango_color_min[1] <= color_promedio[1] <= rango_color_max[1] and
            rango_color_min[2] <= color_promedio[2] <= rango_color_max[2]):
            # If it meets the color range, add it to the filtered list
            circulos_filtrados.append((x, y, r))

    return circulos_filtrados

Función para superponer los círculos en la imagen original. 

In [157]:
def superponer_circulos(imagen, circulos):
    # Create a copy of the image to avoid modifying the original
    imagen_superpuesta = imagen.copy()

    # Check that the list of circles is not empty
    if not circulos:
        print("No circles were detected.")
        return imagen_superpuesta

    # Draw the circles on the image
    for (x, y, r) in circulos:
        cv2.circle(imagen_superpuesta, (x, y), r, (0, 255, 0), 2)  # Green circle
        cv2.circle(imagen_superpuesta, (x, y), 2, (0, 0, 255), 3)  # Circle center in red

    return imagen_superpuesta

Función para obtener radio de la moneda de referencia.

In [158]:
def obtener_radio_referencia(circulos, punto_click):
    for (x, y, r) in circulos:
        distancia = np.sqrt((punto_click[0] - x) ** 2 + (punto_click[1] - y) ** 2)
        if distancia <= r:
            return r
    return None

Función para clasificar las monedas.

In [159]:
def clasificar_monedas(circulos, relacion_escala):

    # Reset coins before classification
    monedas_clasificadas = [0] * 8

    for (x, y, r) in circulos:
        # Calculate the real diameter of the detected coin in millimeters
        diametro_detectado = 2 * r  # Diameter in pixels
        diametro_real = diametro_detectado * relacion_escala

        # Find the coin whose real diameter is closest to the calculated diameter
        diferencia_minima = float('inf')
        valor_asignado = None
        for valor, diametro in diametros_reales.items():
            diferencia = abs(diametro_real - diametro)
            if diferencia < diferencia_minima:
                diferencia_minima = diferencia
                valor_asignado = valor

        # Update the count of coins based on the assigned value
        if valor_asignado == 0.01:
            monedas_clasificadas[0] += 1
        elif valor_asignado == 0.02:
            monedas_clasificadas[1] += 1
        elif valor_asignado == 0.05:
            monedas_clasificadas[2] += 1
        elif valor_asignado == 0.1:
            monedas_clasificadas[3] += 1
        elif valor_asignado == 0.2:
            monedas_clasificadas[4] += 1
        elif valor_asignado == 0.5:
            monedas_clasificadas[5] += 1
        elif valor_asignado == 1.0:
            monedas_clasificadas[6] += 1
        elif valor_asignado == 2.0:
            monedas_clasificadas[7] += 1

    # Update the global `coins` list after classification
    global coins
    coins = monedas_clasificadas

Función que muestra las monedas detectadas y la relación asociada

In [160]:
def mostrar_resultados_deteccion(coins, monedas_reales, relacion_escala):
    nombres_monedas = [
        "Monedas de 1 céntimo",
        "Monedas de 2 céntimos",
        "Monedas de 5 céntimos",
        "Monedas de 10 céntimos",
        "Monedas de 20 céntimos",
        "Monedas de 50 céntimos",
        "Monedas de 1 euro",
        "Monedas de 2 euros"
    ]
    
    print("Resultados de la detección de monedas:")
    print("=====================================")
    for i, cantidad in enumerate(coins):
        if cantidad > 0:
            print(f"{nombres_monedas[i]} detectadas: {cantidad} --- reales: {monedas_reales[i]}")

    print("\nNúmero de monedas detectadas: " + str(sum(coins))+ " ---" + " Número de monedas reales:" + str(sum(monedas_reales)))
    print("Relación de escala utilizada para la detección:")
    print(f"Relación de escala (mm/píxeles): {relacion_escala:.4f}")

Función que muestra la imagen en una ventana.

In [161]:
def mostrar_imagen(imagen, monedas_reales, ancho=1100, alto=600):
    def detectar_color_y_radio(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            # Get the reference coin radius (50 cents)
            radio_referencia = obtener_radio_referencia(circulos_filtrados, (x, y))
            if radio_referencia is not None:
                print(f"Reference coin radius (50 cents): {radio_referencia}")

                # Calculate the detected diameter and the scale ratio
                diametro_detectado = 2 * radio_referencia
                diametro_real_50c = diametros_reales[0.5]
                relacion_escala = diametro_real_50c / diametro_detectado
                print(f"Scale ratio (mm/pixels): {relacion_escala}")

                # Classify the detected coins using the scale ratio
                clasificar_monedas(circulos_filtrados, relacion_escala)

                # Calculate the total money
                total_dinero = moneyCalculate(coins)
                total_dinero_real = moneyCalculate(monedas_reales)
                print(f"Total money in the image: {total_dinero:.2f} euros, Total real: {total_dinero_real:.2f}")
                mostrar_resultados_deteccion(coins, monedas_reales, relacion_escala)
            else:
                print("No reference coin found at the selected point.")

    # Create a fixed size window
    cv2.namedWindow("Imagen con Fondo Blanco", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Imagen con Fondo Blanco", ancho, alto)

    # Assign the callback for the click event
    cv2.setMouseCallback("Imagen con Fondo Blanco", detectar_color_y_radio)

    # Show the image in the window
    cv2.imshow("Imagen con Fondo Blanco", imagen)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Función que permite manejar mejor los datos de entrada.

In [162]:
def ejecutar(imagen, monedas_reales):

    imagen_con_fondo_blanco = convertir_fondo_a_blanco(imagen)
    circulos_detectados  = detectar_circulos(imagen_con_fondo_blanco)

    # Filtrado doradas
    circulos_filtrados.extend(filtrar_por_color(imagen_con_fondo_blanco, circulos_detectados, (20, 0, 0), (180, 170, 255)))
    # Filtrado cobrizas
    circulos_filtrados.extend(filtrar_por_color(imagen_con_fondo_blanco, circulos_detectados, (9, 79, 38), (19, 220, 255)))

    imagen_con_circulos = superponer_circulos(imagen, circulos_filtrados)

    mostrar_imagen(imagen_con_circulos, monedas_reales)


In [166]:
ejecutar(imagen_p2,monedas_p2)
coins = [0] * 8
circulos_filtrados = []

Reference coin radius (50 cents): 84
Scale ratio (mm/pixels): 0.14434523809523808
Total money in the image: 4.07 euros, Total real: 3.01
Resultados de la detección de monedas:
Monedas de 1 céntimo detectadas: 3 --- reales: 3
Monedas de 2 céntimos detectadas: 2 --- reales: 4
Monedas de 5 céntimos detectadas: 4 --- reales: 2
Monedas de 10 céntimos detectadas: 1 --- reales: 1
Monedas de 20 céntimos detectadas: 1 --- reales: 1
Monedas de 50 céntimos detectadas: 1 --- reales: 1
Monedas de 1 euro detectadas: 1 --- reales: 2
Monedas de 2 euros detectadas: 1 --- reales: 0

Número de monedas detectadas: 14 --- Número de monedas reales:14
Relación de escala utilizada para la detección:
Relación de escala (mm/píxeles): 0.1443
